# ONNX tutorial with PyTorch

## Install necessary libraries

Set up the environment and install TurboML's SDK. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
! conda install -qq conda-forge::libstdcxx-ng anaconda::protobuf
! gdown -q 1UG8cmBLdYtEVJZG8Rgz454l722VAt_zE
! unzip -qq turboml.zip -d turboml && pip install -qq turboml/*.whl && rm -rf turboml turboml.zip

Login to your TurboML instance.

In [ ]:
import pandas as pd
import turboml as tb
tb.init(backend_url=BACKEND_URL, api_key=API_KEY)

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install onnx==1.14.1

## PyTorch - Standard Model Training

The following blocks of code define a standard pytorch dataloader, and model training code. This is completely independent of TurboML. 

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import io

In [ ]:
transactions = pd.read_csv("data/transactions.csv").reset_index()
labels = pd.read_csv("data/labels.csv").reset_index()

In [ ]:
joined_df = pd.merge(transactions, labels, on="index", how="right")
joined_df

In [ ]:
X = joined_df.drop("is_fraud", axis=1)
numerical_fields = [
    "transactionAmount",
    "localHour",
    "isProxyIP",
    "digitalItemCount",
    "physicalItemCount",
]

feats = X[numerical_fields]
targets = joined_df["is_fraud"].astype(int)

In [ ]:
class TransactionsDataset(Dataset):
    def __init__(self, feats, targets):
        self.feats = feats
        self.targets = targets

    def __len__(self):
        return len(self.feats)

    def __getitem__(self, idx):
        return {
            "x": torch.tensor(self.feats.iloc[idx], dtype=torch.float),
            "y": torch.tensor(self.targets.iloc[idx], dtype=torch.float),
        }

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(
            64, 2
        )  # Output size is 2 for binary classification (fraud or not fraud)

    def forward(self, x):
        # x --> (batch_size, input_size)
        x = torch.relu(self.fc1(x))
        # x --> (batch_size, 64)
        x = torch.relu(self.fc2(x))
        # x --> (batch_size, 64)
        x = self.fc3(x)
        # x --> (batch_size, 2)
        return x

In [ ]:
model = NeuralNet(input_size=feats.shape[1])
model

In [ ]:
ds = TransactionsDataset(feats, targets)
ds[0]

In [ ]:
train_size = int(0.8 * len(ds))
test_size = len(ds) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(ds, [train_size, test_size])

In [ ]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for data in train_loader:
        inputs = data["x"].float().to(device)
        tars = data["y"].long().to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, tars)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

In [ ]:
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0

with torch.no_grad():
    for data in test_loader:
        inputs = data["x"].float()
        tars = data["y"].long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += tars.size(0)
        total_correct += (predicted == tars).sum().item()

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)

## Export model to ONNX format

Exporting a model to ONNX format depends on the framework. Tutorials for different frameworks can be found at https://github.com/onnx/tutorials#converting-to-onnx-format

In [ ]:
model.eval()
sample_input = torch.randn(1, len(numerical_fields))
buffer = io.BytesIO()
torch.onnx.export(model, sample_input, buffer, export_params=True, verbose=True)
onnx_model_string = buffer.getvalue()

## Create an ONNX model with TurboML

Now that we've converted the model to ONNX format, we can deploy it with TurboML. 

In [ ]:
transactions = tb.PandasDataset(
    dataset_name="transactions_onxx_torch",
    key_field="index",
    dataframe=transactions,
    upload=True,
)
labels = tb.PandasDataset(
    dataset_name="labels_onxx_torch", key_field="index", dataframe=labels, upload=True
)
features = transactions.get_input_fields(numerical_fields=numerical_fields)
label = labels.get_label_field(label_field="is_fraud")

In [ ]:
tb.set_onnx_model("torchmodel", onnx_model_string)
onnx_model = tb.ONNX(model_save_name="torchmodel")

In [ ]:
deployed_model = onnx_model.deploy("onnx_model_torch", input=features, labels=label)

In [ ]:
deployed_model.add_metric("WindowedAUC")

In [ ]:
model_auc_scores = deployed_model.get_evaluation("WindowedAUC")
plt.plot([model_auc_score.metric for model_auc_score in model_auc_scores])